<a href="https://colab.research.google.com/github/oneir0mancer/stable-diffusion-diffusers-colab-ui/blob/main/sd_kandinsky_colab_ui.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kandinsky

In [ ]:
#@title Install dependencies
!pip install 'git+https://github.com/ai-forever/Kandinsky-2.git'
!pip install git+https://github.com/openai/CLIP.git
!mkdir -p outputs/{txt2img,img2img}
!git clone https://github.com/oneir0mancer/stable-diffusion-diffusers-colab-ui.git StableDiffusionUi

In [ ]:
#@title Create model
from kandinsky2 import get_kandinsky2

model = get_kandinsky2('cuda', task_type='text2img', cache_dir='/tmp/kandinsky2', 
                       model_version='2.1', use_flash_attention=False)
sampler = "p_sampler"

In [ ]:
#@title (Optional) Change sampler 
sampler = "p_sampler" #@param ["p_sampler", "ddim_sampler", "plms_sampler"]

## Txt2Img

In [ ]:
#@title Render UI
#@markdown You don't need to run this cell again unless you want to change these settings
save_images = True #@param {type:"boolean"}
display_previewes = True    #@param {type:"boolean"}
#@markdown ---

from StableDiffusionUi.ColabUI.KandinskyUI import KandinskyUI

ui = KandinskyUI()
ui.render()

In [ ]:
#@title Run this cell to generate images
images = ui.generate(model, sampler)

if display_previewes:
    ui.display_image_previews(images)

if save_images:
    for image in results.images:
        image.save(f"outputs/txt2img/{output_index:05}.png")
        print(f"outputs/txt2img/{output_index:05}.png")
        output_index += 1

## Image mixing
(WIP)

In [ ]:
#@title Render UI
import io
import ipywidgets as widgets
from PIL import Image

text_img_a = widgets.Text(placeholder="Path to 1st image")
text_img_b = widgets.Text(placeholder="Path to 2nd image")
slider_img_a = widgets.FloatSlider(max=1, step=0.05)
slider_img_b = widgets.FloatSlider(max=1, step=0.05)
text_prompt_a = widgets.Text(placeholder="Optional 1st prompt")
text_prompt_b = widgets.Text(placeholder="Optional 2nd prompt")
slider_prompt_a = widgets.FloatSlider(max=1, step=0.05)
slider_prompt_b = widgets.FloatSlider(max=1, step=0.05)

width_field = widgets.IntText(value=512, layout=widgets.Layout(width='200px'), description="width: ")
height_field = widgets.IntText(value=768, layout=widgets.Layout(width='200px'), description="height: ")
steps_field = widgets.IntSlider(value=20, min=1, max=100, description="Steps: ")
cfg_field = widgets.FloatSlider(value=7, min=1, max=20, step=0.5, description="CFG: ")

display(widgets.HBox([text_img_a, text_img_b]), widgets.HBox([slider_img_a, slider_img_b]))
display(widgets.HBox([text_prompt_a, text_prompt_b]), widgets.HBox([slider_prompt_a, slider_prompt_b]))
display(widgets.HBox([width_field, height_field]), steps_field, cfg_field)

In [ ]:
#@title Run this cell to generate
def display_image_previews(images):
        wgts = []
        for image in images:
            img_byte_arr = io.BytesIO()
            image.save(img_byte_arr, format='PNG')
            wgt = widgets.Image(
                value=img_byte_arr.getvalue(),
                width=image.width/2,
                height=image.height/2,
            )
            wgts.append(wgt)
        display(widgets.HBox(wgts))

img_a = Image.open(text_img_a.value)
img_b = Image.open(text_img_b.value)
images_texts = [text_prompt_a.value, img_a, img_b, text_prompt_b.value]
weights = [slider_prompt_a.value, slider_img_a.value, slider_img_b.value, slider_prompt_b.value]

images = model.mix_images(
    images_texts, 
    weights, 
    num_steps=steps_field.value,
    batch_size=1, 
    guidance_scale=cfg_field.value,
    h=height_field.value, w=width_field.value,
    sampler=sampler, 
    prior_cf_scale=4,
    prior_steps="5"
)
display_image_previews([img_a, images[0], img_b])